# Create Plazi Collectors Data Set and Match Names to WikiData

Create a data set of collectors recorded by Plazi:

- see <https://tb.plazi.org/GgServer/srsStats> section “Materials Citation Data”
- then select the data (columns) of interest, and then below on section **Fields to Use in Statistics** you can alter the output
    - choose **Operation** “show individual values”
    - filter values at **Filter on Values**
    - set the limit to e.g. 5 to see what data you would get
    - below you can get the download link to the data format you get offered there

# Example Data

| Field Name | Filter on Values |
|-|-|
| Collector Name          | >0 |
| GBIF Occurrence ID      | !0 |
| Collecting Month        |    |
| Collecting Year         |    |
| Collecting Decade       |    |
| Collecting Date         |    |
| Materials Citation UUID |    |

```bash
# added filter: gbifOccurrenceId → !0
# added filter: collector → >0 (seems to give the non empty collector names)
filename="plazi-stats_numberOfTreatments_gbifOccurrenceId-not0_date_decade_year_month_collector-gt0_$(date '+%Y%m%d').tsv"
wget --output-document="${filename}" \
'https://tb.plazi.org/GgServer/srsStats/stats?outputFields=matCit.id+matCit.gbifOccurrenceId+matCit.date+matCit.decade+matCit.year+matCit.month+matCit.collector&groupingFields=matCit.id+matCit.gbifOccurrenceId+matCit.date+matCit.decade+matCit.year+matCit.month+matCit.collector&FP-matCit.gbifOccurrenceId=!0&FP-matCit.collector=%3E0&format=TSV'

cat "${filename}" | wc -l
# 417402 minus 1 record (=column header)

{ head -n 5 "${filename}"; echo "..."; tail -n 5 "${filename}"; } | column --table --separator $'\t' | sed 's@^@  # @;'
  # DocCount  MatCitId                          MatCitGbifOccurrenceId  MatCitDate  MatCitDecade  MatCitYear  MatCitMonth  MatCitCollector
  # 1         78F03CF8FFE2FFE5C0C4F883FE73F8B4  3419301320                          0             0           0            1888 - 1890 & Morong, T.
  # 1         78F03CF8FFE5FFE2C187FB83FD0AFB94  3419301397                          0             0           0            1914 & Chodat, R.
  # 1         1FFD3CFF806D3D11C410027311B3FEAC  4012799597              1980-09-19  1980          1980        9            1980 - Sino- American Botanical Expedition
  # 1         AFA17A73FFA8F2414DA6F9AB94DCF942  3466701331                          0             0           0            20. 8.201 3 & Delage, A.
  # ...                                                                                                                    
  # 1         3B7F3CD7FFEDFFF5FB68FCBD4061FCB8  3072658352              2017-07-05  2010          2017        7            Z. Z. Xia
  # 1         3B5C3CD3FF9FFFACFCCB2B09BAD0FE79  1699618906              2002-06-25  2000          2002        6            Z. Z. Yang
  # 1         B5B23CA2C006FF87FB6FF9CBFA17F94A  2028140173              2009-08-18  2000          2009        8            Z. Z. Yang
  # 1         3B063C92F16FFF93DA9FFC4DFEDB1D0B  3866542316              2015-06-08  2010          2015        6            ZZ Zhang
  # 1         3B7C3CAD6B18FFBCADDEFA01FE543FE5  3034555558              1956-06-20  1950          1956        6            А. Schnitnikov
```



In [2]:
import json
import requests
import pandas as pd
import time

# https://tb.plazi.org/GgServer/srsStats/stats?
#   outputFields=matCit.id+matCit.gbifOccurrenceId+matCit.date+matCit.decade+matCit.year+matCit.month+matCit.collector
#   &
#   groupingFields=matCit.id+matCit.gbifOccurrenceId+matCit.date+matCit.decade+matCit.year+matCit.month+matCit.collector
#   &
#   FP-matCit.gbifOccurrenceId=!0
#   &
#   FP-matCit.collector=%3E0
#   &
#   format=TSV
url = 'https://tb.plazi.org/GgServer/srsStats/stats'
params = [
    ('outputFields',   'matCit.id matCit.gbifOccurrenceId matCit.date matCit.decade matCit.year matCit.month matCit.collector'),
    ('groupingFields', 'matCit.id matCit.gbifOccurrenceId matCit.date matCit.decade matCit.year matCit.month matCit.collector'),
    ('FP-matCit.gbifOccurrenceId', '!0'),
    ('FP-matCit.collector', '>0'),
    ('format', 'JSON')
]

start_time = time.time()
print("Send data request to" , url)

response = requests.get(url, params)
dict = response.json()
collectors = dict['data']

print("Response of %s came in %s seconds (HTTP-code: %s)" % (
    url, 
    (time.time() - start_time), 
    response.status_code)
)

start_time = time.time()
print("Normalize JSON data with pandas …")

df = pd.json_normalize(collectors)

print("Normalization took %s seconds" % (time.time() - start_time) )

print("Print data sample …")
df



Send data request to https://tb.plazi.org/GgServer/srsStats/stats
Response of https://tb.plazi.org/GgServer/srsStats/stats came in 12.322981357574463 seconds (HTTP-code: 200)
Normalize JSON data with pandas …
Normalization took 2.3167567253112793 seconds
Print data sample …


,DocCount,MatCitId,MatCitGbifOccurrenceId,MatCitDate,MatCitDecade,MatCitYear,MatCitMonth,MatCitCollector
0,1,78F03CF8FFE2FFE5C0C4F883FE73F8B4,3419301320,,0,0,0,"1888 - 1890 & Morong, T."
1,1,78F03CF8FFE5FFE2C187FB83FD0AFB94,3419301397,,0,0,0,"1914 & Chodat, R."
2,1,1FFD3CFF806D3D11C410027311B3FEAC,4012799597,1980-09-19,1980,1980,9,1980 - Sino- American Botanical Expedition
3,1,AFA17A73FFA8F2414DA6F9AB94DCF942,3466701331,,0,0,0,"20. 8.201 3 & Delage, A."
4,1,87ADD56BFF8DFF9BFBA0164C25E5FA86,3467693310,,0,0,0,"20. IX. 1957 & fr., Service Forestier"
...,...,...,...,...,...,...,...,...
423968,1,3B7F3CD7FFEDFFF5FB68FCBD4061FCB8,3072658352,2017-07-05,2010,2017,7,Z. Z. Xia
423969,1,3B5C3CD3FF9FFFACFCCB2B09BAD0FE79,1699618906,2002-06-25,2000,2002,6,Z. Z. Yang
423970,1,B5B23CA2C006FF87FB6FF9CBFA17F94A,2028140173,2009-08-18,2000,2009,8,Z. Z. Yang
423971,1,3B063C92F16FFF93DA9FFC4DFEDB1D0B,3866542316,2015-06-08,2010,2015,6,ZZ Zhang


In [3]:
list(df.columns)

['DocCount',
 'MatCitId',
 'MatCitGbifOccurrenceId',
 'MatCitDate',
 'MatCitDecade',
 'MatCitYear',
 'MatCitMonth',
 'MatCitCollector']

In [4]:
# move 'MatCitCollector' to be the first column (prepare parsing names for bin/agent_parse4tsv.rb: collectors in the 1st column)
col = df.pop("MatCitCollector")
df.insert(0, col.name, col)
df

,MatCitCollector,DocCount,MatCitId,MatCitGbifOccurrenceId,MatCitDate,MatCitDecade,MatCitYear,MatCitMonth
0,"1888 - 1890 & Morong, T.",1,78F03CF8FFE2FFE5C0C4F883FE73F8B4,3419301320,,0,0,0
1,"1914 & Chodat, R.",1,78F03CF8FFE5FFE2C187FB83FD0AFB94,3419301397,,0,0,0
2,1980 - Sino- American Botanical Expedition,1,1FFD3CFF806D3D11C410027311B3FEAC,4012799597,1980-09-19,1980,1980,9
3,"20. 8.201 3 & Delage, A.",1,AFA17A73FFA8F2414DA6F9AB94DCF942,3466701331,,0,0,0
4,"20. IX. 1957 & fr., Service Forestier",1,87ADD56BFF8DFF9BFBA0164C25E5FA86,3467693310,,0,0,0
...,...,...,...,...,...,...,...,...
423968,Z. Z. Xia,1,3B7F3CD7FFEDFFF5FB68FCBD4061FCB8,3072658352,2017-07-05,2010,2017,7
423969,Z. Z. Yang,1,3B5C3CD3FF9FFFACFCCB2B09BAD0FE79,1699618906,2002-06-25,2000,2002,6
423970,Z. Z. Yang,1,B5B23CA2C006FF87FB6FF9CBFA17F94A,2028140173,2009-08-18,2000,2009,8
423971,ZZ Zhang,1,3B063C92F16FFF93DA9FFC4DFEDB1D0B,3866542316,2015-06-08,2010,2015,6


## Write the Output Data



In [32]:
import os
from datetime import datetime

if not os.path.exists('data'):
    print("Make data directory for saving …")
    os.makedirs('data')

this_output_file=os.path.join(
    "data", ("plazi_GbifOccurrenceId_CitCollector_%s.tsv" % datetime.today().strftime('%Y%m%d'))
)

df.to_csv(this_output_file
          , sep='\t'
          ,index=False # skip the index
    # , header=["custom_colname_1", "custom_colname_2", "…"] # could rewrite header labels
)

print("Wrote data results into into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Write data results into into data/plazi_GbifOccurrenceId_CitCollector_20230719.tsv (34790 kB)


## Parse Collector Names

Now you can parse the names with dwcagent, if the collector names are in the first column:

```bash
cd bin
ruby agent_parse4tsv.rb \
  --input ../data/plazi_GbifOccurrenceId_CitCollector_20230719.tsv \
  --output ../data/plazi_GbifOccurrenceId_CitCollector_20230719_parsed.tsv

# or check also running time of the parsing script with `time command`

time ruby agent_parse4tsv.rb \
  --input ../data/plazi_GbifOccurrenceId_CitCollector_20230719.tsv \
  --output ../data/plazi_GbifOccurrenceId_CitCollector_20230719_parsed.tsv
# real    5m2,451s
# user    2m38,644s
# sys     2m4,709s
```

## Load WikiData Names and Parsed Collector Data

This procedure follows Niels Klazenga’s `match_names_to_wikidata_items.ipynb` (<https://github.com/nielsklazenga/avh-collectors/blob/47c3374f02bea4064b1c6708d79bcd9ba55a08a0/match_names_to_wikidata_items.ipynb>).

Use [`create_wikidata_datasets_botanists.ipynb`](create_wikidata_datasets_botanists.ipynb) to generate the data of botanist of WikiData first, then load those data to prepare the match of your data:

In [13]:
import pandas as pd
wikidata = pd.read_csv("data/wikidata_persons_botanists_20230703_1352.csv", index_col=0, low_memory=False)

wikidata.head()

,item,itemLabel,surname,initials,canonical_string,orcid,viaf,isni,harv,ipni,abbr,bionomia_id,yob,yod,wyb,wye
0,http://www.wikidata.org/entity/Q66612,Friedrich August Marschall von Bieberstein,Bieberstein,F.A.M.v.,"Bieberstein, F.A.M.v.",NaN,43340073,0000 0001 1630 5464,1373.0,6129-1,M.Bieb.,Q66612,1768.0,1826.0,NaN,NaN
1,http://www.wikidata.org/entity/Q66934,Hans Hermann Behr,Behr,H.H.,"Behr, H.H.",NaN,20328622,0000 0001 1604 8680,42741.0,619-1,Behr,Q66934,1818.0,1904.0,NaN,NaN
2,http://www.wikidata.org/entity/Q66661,Jacob Christian Schäffer,Schäffer,J.C.,"Schäffer, J.C.",NaN,47016953,0000 0000 8343 3899,1101.0,12818-1,Schaeff.,NaN,1718.0,1790.0,NaN,NaN
3,http://www.wikidata.org/entity/Q67003,Johann Friedrich Klotzsch,Klotzsch,J.F.,"Klotzsch, J.F.",NaN,20426762,0000 0001 1749 2732,135.0,4855-1,Klotzsch,Q67003,1805.0,1860.0,NaN,NaN
4,http://www.wikidata.org/entity/Q66322,Franz Anton Menge,Menge,F.A.,"Menge, F.A.",NaN,59847236,0000 0001 1653 0899,73782.0,23266-1,Menge,NaN,1808.0,1880.0,NaN,NaN


In [14]:
# create the test data set of WikiData data
# group by canonical name/string, count douplicated names
wd_matchtest = wikidata.groupby('canonical_string').agg({'item': ['count']}).reset_index()
# TODO AP: meaning of wd_matchtest + count for merge later on? 

wd_matchtest.tail()

# colls = list(wikidata.columns)

,canonical_string,item
,,count
61296,"Șerbanescu, I.",1
61297,"Ștefureac, T.",1
61298,"Țopa, E.",1
61299,"Ḥalwaǧī, R.",1
61300,"Ḳushnir, Ṭ.",1


In [23]:
# unique names parsed already by ruby gem package: dwcagent

collectors = pd.read_csv("data/plazi_GbifOccurrenceId_CitCollector_20230719_parsed.tsv", sep="\t", low_memory=False)

collectors.dropna(subset=['family'], inplace=True) # remove where family was NA, e.g. from originally «??» aso.
collectors

,family,given,suffix,particle,dropping_particle,nick,appellation,title,DocCount,MatCitId,MatCitGbifOccurrenceId,MatCitDate,MatCitDecade,MatCitYear,MatCitMonth
0,Chodat,R.,NaN,NaN,NaN,NaN,NaN,NaN,1,78F03CF8FFE5FFE2C187FB83FD0AFB94,3419301397,NaN,0,0,0
1,Mayo,NaN,NaN,de,NaN,NaN,NaN,NaN,1,31ADD85BA138FFE3FF45A111FB90F6CB,3421410670,2001-01-18,2000,2001,1
2,Garcete,B.,NaN,NaN,NaN,NaN,NaN,NaN,1,31ADD85BA138FFE3FF45A111FB90F6CB,3421410670,2001-01-18,2000,2001,1
5,Virginia,A,NaN,NaN,NaN,NaN,NaN,NaN,1,3B553CEFFFD5FFC6FF34F4ABFD46FEDA,3333037406,2019-08-01,2010,2019,8
6,Virginia,A,NaN,NaN,NaN,NaN,NaN,NaN,1,3B553CEFFFD6FFC6FBC8F2A7FAA2FDF8,3333037676,2019-08-02,2010,2019,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607462,Xia,Z.Z.,NaN,NaN,NaN,NaN,NaN,NaN,1,3B7F3CD7FFEDFFF5FB68FCBD4061FCB8,3072658352,2017-07-05,2010,2017,7
607463,Yang,Z.Z.,NaN,NaN,NaN,NaN,NaN,NaN,1,3B5C3CD3FF9FFFACFCCB2B09BAD0FE79,1699618906,2002-06-25,2000,2002,6
607464,Yang,Z.Z.,NaN,NaN,NaN,NaN,NaN,NaN,1,B5B23CA2C006FF87FB6FF9CBFA17F94A,2028140173,2009-08-18,2000,2009,8
607465,Zhang,Z.Z.,NaN,NaN,NaN,NaN,NaN,NaN,1,3B063C92F16FFF93DA9FFC4DFEDB1D0B,3866542316,2015-06-08,2010,2015,6


#### Check Composition of Parsed Collector Data

In [16]:
# test particle for NA values (perhaps particle is the most important)
test_collectors = collectors.loc[(collectors.particle.isna() == False)]
print("names with particle (%s records)…" % len(test_collectors.index))
test_collectors.head()

names with particle (19675 records)…


,family,given,suffix,particle,dropping_particle,nick,appellation,title,DocCount,MatCitId,MatCitGbifOccurrenceId,MatCitDate,MatCitDecade,MatCitYear,MatCitMonth
1,Mayo,NaN,NaN,de,NaN,NaN,NaN,NaN,1,31ADD85BA138FFE3FF45A111FB90F6CB,3421410670,2001-01-18,2000,2001,1
270,A. A. Girault,G.,NaN,as,NaN,NaN,NaN,NaN,1,E4E73CEFE566FFFE6C4A0CFC1E2D5BF7,3743912342,1909-08-25,1900,1909,8
271,A. A. Girault,G.,NaN,as,NaN,NaN,NaN,NaN,1,E4E73CEFE566FFFE6C040CD1191C5BD2,3743912408,1910-07-01,1910,1910,7
1066,Grave,S.,NaN,De,NaN,NaN,NaN,NaN,1,84B80478FFF8FFBFFE9DFC0A2796D4CB,3026647302,2019-09-25,2010,2019,9
1070,Grave,S.,NaN,De,NaN,NaN,NaN,NaN,1,84B80478FFF8FFBFFF2FFC2E278FD337,3026647304,2019-09-25,2010,2019,9


In [17]:
# test suffix for NA values
test_collectors = collectors.loc[(collectors.suffix.isna() == False)]
print("names with suffix (%s records)…" % len(test_collectors.index))
test_collectors.head()

names with suffix (750 records)…


,family,given,suffix,particle,dropping_particle,nick,appellation,title,DocCount,MatCitId,MatCitGbifOccurrenceId,MatCitDate,MatCitDecade,MatCitYear,MatCitMonth
731,Braul,A.,Jr.,NaN,NaN,NaN,NaN,NaN,1,3B1AA47D24368929DD80AAE03C1925BF,1977784106,NaN,0,0,0
733,Braul,A.,Jr.,NaN,NaN,NaN,NaN,NaN,1,3B1AA47D24368929DE4DAA283A792583,1977784095,1992-08-04,1990,1992,8
735,Braul,A.,Jr.,NaN,NaN,NaN,NaN,NaN,1,3B1AA47D24368929D9F1AAE33CC725DB,1977783800,1993-08-02,1990,1993,8
3382,Creek,Abingdon,SR,NaN,NaN,NaN,NaN,NaN,1,3B7E8260FFAB730BC762FF05FECFECA1,4089505452,2011-04-18,2010,2011,4
3543,Kingman,Abner,Jr.,NaN,NaN,NaN,NaN,NaN,1,FC407A7EAD115D2C3F4EA0A25F0B203C,1058480287,1992-07-21,1990,1992,7


In [18]:
test_collectors = collectors.loc[(collectors.dropping_particle.isna() == False)]
print("names with dropping_particle (%s records)…" % len(test_collectors.index))
test_collectors.head()

names with dropping_particle (0 records)…


,family,given,suffix,particle,dropping_particle,nick,appellation,title,DocCount,MatCitId,MatCitGbifOccurrenceId,MatCitDate,MatCitDecade,MatCitYear,MatCitMonth


In [19]:
test_collectors = collectors.loc[(collectors.appellation.isna() == False)]
print("names with appellation (%s records)…" % len(test_collectors.index))
test_collectors.head()

names with appellation (268 records)…


,family,given,suffix,particle,dropping_particle,nick,appellation,title,DocCount,MatCitId,MatCitGbifOccurrenceId,MatCitDate,MatCitDecade,MatCitYear,MatCitMonth
22457,Longfield,C.,NaN,NaN,NaN,NaN,Miss,NaN,1,15897E5611E2679F420F33806B931C6B,4127603396,1927-05-30,1920,1927,5
28788,Farren,NaN,NaN,NaN,NaN,NaN,Mrs.,NaN,1,3B2D3C96831E9D43FF4D4018FB442853,1585880133,1809,1800,1849,9
33321,Carr,NaN,NaN,NaN,NaN,NaN,Mrs,NaN,1,3B693415875AFFDF80BC40A8FC707060,3399886349,1957-03,1950,1957,3
83286,Barrett,F.,NaN,NaN,NaN,NaN,Miss,NaN,1,3B77D118FFF6E35AE4DBFD1BFC96FD26,2610429310,NaN,0,0,0
89747,Cooper,NaN,NaN,NaN,NaN,NaN,Mr,NaN,1,3B61A0705C0EFFC86382FBA0FE73FA38,3328594336,1802-12-08,1800,1802,12


Compile `canonical_string...` for the collector data we will later match the WikiData names with:

In [24]:
collectors['canonical_string_collector_parsed'] = (
  # use collectors.family only where given name has NA values, otherwise use family name + given name
  collectors.family.where(
      # condition
      collectors.given.isna(),
      # any other TODO improve the combined name for canonical_string_collector_parsed if any of the other dwc_parsed fields is not NaN
      # other= collectors.family + ", " + collectors.given 
      other= (collectors.family + ", " + collectors.given) \
        if any(collectors.particle.isna()) \
        else collectors.particle + " " + collectors.family \
         + ", " + collectors.given
  )
)

# move 'canonical_string_collector_parsed' before DocCount for readablity
col = collectors.pop("canonical_string_collector_parsed")
collectors.insert(collectors.columns.get_loc('DocCount'), col.name, col)
collectors.tail()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,DocCount,MatCitId,MatCitGbifOccurrenceId,MatCitDate,MatCitDecade,MatCitYear,MatCitMonth
607462,Xia,Z.Z.,NaN,NaN,NaN,NaN,NaN,NaN,"Xia, Z.Z.",1,3B7F3CD7FFEDFFF5FB68FCBD4061FCB8,3072658352,2017-07-05,2010,2017,7
607463,Yang,Z.Z.,NaN,NaN,NaN,NaN,NaN,NaN,"Yang, Z.Z.",1,3B5C3CD3FF9FFFACFCCB2B09BAD0FE79,1699618906,2002-06-25,2000,2002,6
607464,Yang,Z.Z.,NaN,NaN,NaN,NaN,NaN,NaN,"Yang, Z.Z.",1,B5B23CA2C006FF87FB6FF9CBFA17F94A,2028140173,2009-08-18,2000,2009,8
607465,Zhang,Z.Z.,NaN,NaN,NaN,NaN,NaN,NaN,"Zhang, Z.Z.",1,3B063C92F16FFF93DA9FFC4DFEDB1D0B,3866542316,2015-06-08,2010,2015,6
607466,Schnitnikov,А.,NaN,NaN,NaN,NaN,NaN,NaN,"Schnitnikov, А.",1,3B7C3CAD6B18FFBCADDEFA01FE543FE5,3034555558,1956-06-20,1950,1956,6


In [ ]:
# collectors=collectors.add_suffix('_parsed') \
#  if not any(col.endswith("_parsed") for col in list(collectors.columns))

### Set Up the Text Search

See https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536

The ngrams function is used as an analyzer in the text search later.

In [25]:
wd_matchtest['canonical_string'].at[0]

'(-Walraevens), O.H.'

In [26]:
import re
!pip install ftfy # amazing text cleaning for decode issues..
from ftfy import fix_text

def ngrams(string, n=3):
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title() # normalise case - capital at start of each word
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single
    string = ' '+ string +' ' # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]


print("Example from name:", ngrams('Klazenga, N.'))
print("Example from collectors:", ngrams(collectors["canonical_string_collector_parsed"].at[1])) 
print("Example from match-test:", ngrams(wd_matchtest['canonical_string'].at[1]))

Defaulting to user installation because normal site-packages is not writeable
Example from name: [' Kl', 'Kla', 'laz', 'aze', 'zen', 'eng', 'nga', 'ga ', 'a N', ' N ']
Example from collectors: [' Ma', 'May', 'ayo', 'yo ']
Example from match-test: [' 18', '183', '835', '35 ', '5 1', ' 19', '190', '906', '06 ', '6 G', ' Ga', 'Gaf', 'afe', 'fe ']


Vectorize Wikidata names. Background: We use an information retrieval technique (Term Frequency — Inverse Document Frequency, blog [towardsdatascience.com/tf-idf-explained…](https://towardsdatascience.com/tf-idf-explained-and-python-sklearn-implementation-b020c5e83275)) for matching the source names with WikiData names, for that a calculated dinsance measure of the name match will help to match similar names and distinguish names that are rather no match. In general see also https://scikit-learn.org, https://pypi.org/project/scikit-learn/. 

Convert a collection of raw documents to a matrix of TF-IDF features:

In [27]:
# pip install --upgrade scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

wikidata_names = wd_matchtest['canonical_string']

# vectorize wikidata names
print('Vectorizing data. This may take a while...')
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
tfidf_vector_data = vectorizer.fit_transform(wikidata_names)
print('Vectorizing completed: Created a matrix of TF-IDF featurs')


Vectorizing data. This may take a while...
Vectorizing completed: Created a matrix of TF-IDF featurs


Set up the function that performs the nearest neighbour matches...

In [28]:
from sklearn.neighbors import NearestNeighbors

nbrs_data = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf_vector_data) 
# tfidf_vector_data contains the vectorized wikidata names from the previous step

# matching query
def getNearestN(query):
    queryTFIDF_ = vectorizer.transform(query)
    distances, indices = nbrs_data.kneighbors(queryTFIDF_)
    return distances, indices


### Perform the Matching

Perform the nearest neighbour (NN) matches on the (Plazi) collector names and create a data frame with matches... (can take 10 to 30 minutes)

In [29]:
collectors_names = set(collectors['canonical_string_collector_parsed'].values) 
  # convert list to set for better performance

import time
start = time.time()
print('Getting nearest neighbours...')
distances, indices = getNearestN(collectors_names)
duration = time.time() - start
print('Completed in:', duration, 's')

collectors_names = list(collectors_names) # convert back to list

print('Finding matches...')
matches = []
for i,j in enumerate(indices):
    temp = [collectors_names[i], wd_matchtest.values[j][0][0], round(distances[i][0],2)]
    matches.append(temp)

duration = time.time() - start
print('Building matches data frame:', duration, 's')  
matches = pd.DataFrame(
    matches, 
    columns=['namematch_collector','namematch_wikidata','namematch_distance']
)

duration = time.time() - start
print('Done:', duration, 's') 

matches = matches.sort_values(['namematch_distance'])
matches = matches.reset_index()

matches.head()

Getting nearest neighbours...
Completed in: 812.5065448284149 s
Finding matches...
Building matches data frame: 1376.0338099002838 s
Done: 1376.1084852218628 s


,index,namematch_collector,namematch_wikidata,namematch_distance
0,71291,"Moon, H.","Moon, H.",0.0
1,89321,"Taylor, E.","Taylor, E.",0.0
2,89323,"Gussone, G.","Gussone, G.",0.0
3,41926,"Oberleitner, F.","Oberleitner, F.",0.0
4,41923,"Hubbs, C.L.","Hubbs, C.L.",0.0


### Create Output Results

Combine the matches data frame back to the (Plazi) collectors and Wikidata items …

In [30]:
# join matches data frame back to source collectors  dataframe 
collectors_matches = pd.merge(
    collectors, matches, 
    left_on='canonical_string_collector_parsed', right_on='namematch_collector'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

collectors_matches.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,DocCount,MatCitId,MatCitGbifOccurrenceId,MatCitDate,MatCitDecade,MatCitYear,MatCitMonth,index,namematch_collector,namematch_wikidata,namematch_distance
0,Chodat,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Chodat, R.",1,78F03CF8FFE5FFE2C187FB83FD0AFB94,3419301397,NaN,0,0,0,45577,"Chodat, R.","Chodat, R.H.",0.53
1,Chodat,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Chodat, R.",1,88A73C93FFA7664B5BDE97A93F63FADE,3416963306,NaN,0,0,0,45577,"Chodat, R.","Chodat, R.H.",0.53
2,Chodat,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Chodat, R.",1,88A73C93FFA7664B592397123F00FB55,3416963307,1914,1910,1914,0,45577,"Chodat, R.","Chodat, R.H.",0.53
3,Chodat,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Chodat, R.",1,78F03CF8FFE5FFE2C045FC03FE70FBD4,3419301305,1914,1910,1914,0,45577,"Chodat, R.","Chodat, R.H.",0.53
4,Chodat,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Chodat, R.",1,78F03CF8FFEBFFECC068FE83FE71FE94,3419301322,1914,1910,1914,0,45577,"Chodat, R.","Chodat, R.H.",0.53


Save the results...

In [33]:
from datetime import datetime
import os
if not os.path.exists('data'):
    os.makedirs('data')

this_output_file='data/plazi_collectors_matches_wikidata-botanists_%s.csv' % (
    # "20230531_1156"
    datetime.today().strftime('%Y%m%d') # '%Y%m%d_%H%M'
)

collectors_matches.to_csv(this_output_file)

print(
    "Wrote matches of collector names into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
) 

Write matches of collector names into data/plazi_collectors_matches_wikidata-botanists_20230719.csv (79876 kB)


### Aggregate Matched Data

Aggregate now the data, if multiple names are found … aso. and join multiple results by “…|…”

In [34]:
# link counts of wikidata items with canonical name string
# collectors_matches_g1 = pd.merge(collectors_matches, wd_matchtest, 
#                                  left_on='matched_name', right_on='canonical_string')
collectors_matches_g1 = pd.merge(
    collectors_matches, wikidata, 
    left_on='namematch_wikidata', right_on='canonical_string'
)
collectors_matches_g1.rename(columns = {list(collectors_matches_g1)[-1]: 'item_count'}, inplace=True)

# link wikidata items with canonical name string (pipe separated if more than one)
print('Aggregate WD item (multiple data items found, joined by “…|…”) ...')
wikidata_uniq_items = wikidata.groupby(['canonical_string'])['item'].apply('|'.join).reset_index()
print('Done.')

collectors_matches_g2 = pd.merge(# no merging unique WikiData names to collectors
    collectors_matches_g1, wikidata_uniq_items, 
    left_on='namematch_wikidata', right_on='canonical_string'
    , suffixes=('__grp_by_itemcount', '__grp_by_item') 
      # append to left-data, right-data only when identical column names occur
)
collectors_matches_g2.rename(columns = {list(collectors_matches_g2)[-1]: 'items'}, inplace=True)

# link wikidata items with canonical name string (pipe separated if more than one)
print('Aggregate WD itemLabel (multiple names found, joined by “…|…”) ...')
wikidata_uniq_itemlabels = wikidata.groupby(['canonical_string'])['itemLabel'].apply('|'.join).reset_index()
print('Done.')

collectors_matches_g3 = pd.merge(
    collectors_matches_g2, wikidata_uniq_itemlabels, 
    left_on='namematch_wikidata', right_on='canonical_string'
    , suffixes=('__grp_by_items', '__grp_by_itemlabel') 
      # append to left-data, right-data only when identical column names occur
)

collectors_matches_g3.rename(columns = {list(collectors_matches_g3)[-1]: 'item_labels'}, inplace=True)


Aggregate WD item (multiple data items found, joined by “…|…”) ...
Done.
Aggregate WD itemLabel (multiple names found, joined by “…|…”) ...
Done.


Prepare data to save later on …

In [35]:
collectors_matches_group = collectors_matches_g3

print(list(collectors_matches_group.columns))
# from merge: _x means from left column, _y means from right column

# in BASH fold text long lines; echo "${text}" | fold --spaces | sed 's@^@# @'
# ['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick', 
# 'appellation', 'title', 'DocCount', 'MatCitId', 'MatCitGbifOccurrenceId', 
# 'MatCitDate', 'MatCitDecade', 'MatCitYear', 'MatCitMonth', 
# 'canonical_string_collector_parsed', 'index', 'namematch_collector', 
# 'namematch_wikidata', 'namematch_distance', 'item__grp_by_itemcount', 
# 'itemLabel__grp_by_items', 'surname', 'initials', 
# 'canonical_string__grp_by_itemcount', 'orcid', 'viaf', 'isni', 'harv', 'ipni', 
# 'abbr', 'bionomia_id', 'yob', 'yod', 'wyb', 'item_count', 
# 'canonical_string__grp_by_item', 'items', 'canonical_string', 'item_labels']

['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick', 'appellation', 'title', 'canonical_string_collector_parsed', 'DocCount', 'MatCitId', 'MatCitGbifOccurrenceId', 'MatCitDate', 'MatCitDecade', 'MatCitYear', 'MatCitMonth', 'index', 'namematch_collector', 'namematch_wikidata', 'namematch_distance', 'item__grp_by_itemcount', 'itemLabel__grp_by_items', 'surname', 'initials', 'canonical_string__grp_by_itemcount', 'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod', 'wyb', 'item_count', 'canonical_string__grp_by_item', 'items', 'canonical_string', 'item_labels']


In [36]:
collectors_matches_group.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,DocCount,...,abbr,bionomia_id,yob,yod,wyb,item_count,canonical_string__grp_by_item,items,canonical_string,item_labels
0,Chodat,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Chodat, R.",1,...,Chodat,NaN,1865.0,1934.0,NaN,NaN,"Chodat, R.H.",http://www.wikidata.org/entity/Q2613173,"Chodat, R.H.",Robert Hippolyte Chodat
1,Chodat,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Chodat, R.",1,...,Chodat,NaN,1865.0,1934.0,NaN,NaN,"Chodat, R.H.",http://www.wikidata.org/entity/Q2613173,"Chodat, R.H.",Robert Hippolyte Chodat
2,Chodat,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Chodat, R.",1,...,Chodat,NaN,1865.0,1934.0,NaN,NaN,"Chodat, R.H.",http://www.wikidata.org/entity/Q2613173,"Chodat, R.H.",Robert Hippolyte Chodat
3,Chodat,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Chodat, R.",1,...,Chodat,NaN,1865.0,1934.0,NaN,NaN,"Chodat, R.H.",http://www.wikidata.org/entity/Q2613173,"Chodat, R.H.",Robert Hippolyte Chodat
4,Chodat,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Chodat, R.",1,...,Chodat,NaN,1865.0,1934.0,NaN,NaN,"Chodat, R.H.",http://www.wikidata.org/entity/Q2613173,"Chodat, R.H.",Robert Hippolyte Chodat


In [37]:
# Remove superfluous columns 
# TODO check WARNING: A value is trying to be set on a copy of a slice from a DataFrame
# TODO check duplicates
collectors_matches_group = collectors_matches_g3[
    ['family', 'given', 'canonical_string_collector_parsed', 
    'namematch_collector', 'namematch_wikidata', 'namematch_distance','MatCitDate',
    'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod', 'wyb',
    'items', 'canonical_string', 'item_labels']
]
# collectors_matches_group = collectors_matches_g3
collectors_matches_group.sort_values(by=['namematch_distance', 'canonical_string_collector_parsed','MatCitDate'], inplace=True)
collectors_matches_group.drop_duplicates(inplace=True)
collectors_matches_group.head()

/tmp/ipykernel_33153/2205490725.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors_matches_group.sort_values(by=['namematch_distance', 'canonical_string_collector_parsed','MatCitDate'], inplace=True)
/tmp/ipykernel_33153/2205490725.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors_matches_group.drop_duplicates(inplace=True)


,family,given,canonical_string_collector_parsed,namematch_collector,namematch_wikidata,namematch_distance,MatCitDate,orcid,viaf,isni,harv,ipni,abbr,bionomia_id,yob,yod,wyb,items,canonical_string,item_labels
609417,Aagaard,K.,"Aagaard, K.","Aagaard, K.","Aagaard, K.",0.0,18.VII.198,NaN,273390791,NaN,NaN,NaN,NaN,Q55216516,1947.0,NaN,NaN,http://www.wikidata.org/entity/Q55216516,"Aagaard, K.",Kaare Aagaard
609418,Aagaard,K.,"Aagaard, K.","Aagaard, K.","Aagaard, K.",0.0,1986-07-01,NaN,273390791,NaN,NaN,NaN,NaN,Q55216516,1947.0,NaN,NaN,http://www.wikidata.org/entity/Q55216516,"Aagaard, K.",Kaare Aagaard
19677,Aarvik',L.,"Aarvik', L.","Aarvik', L.","Aarvik, L.",0.0,1992-03-25,0000-0002-0112-8837,14511016,0000 0000 4817 3374,NaN,NaN,NaN,0000-0002-0112-8837,1954.0,NaN,NaN,http://www.wikidata.org/entity/Q17114254|http:...,"Aarvik, L.",Leif Aarvik|Lars Aarvik
19678,Aarvik',L.,"Aarvik', L.","Aarvik', L.","Aarvik, L.",0.0,1992-03-25,NaN,NaN,NaN,NaN,NaN,NaN,Q106823278,1892.0,1981.0,NaN,http://www.wikidata.org/entity/Q17114254|http:...,"Aarvik, L.",Leif Aarvik|Lars Aarvik
19669,Aarvik,L.,"Aarvik, L.","Aarvik, L.","Aarvik, L.",0.0,1955-01,0000-0002-0112-8837,14511016,0000 0000 4817 3374,NaN,NaN,NaN,0000-0002-0112-8837,1954.0,NaN,NaN,http://www.wikidata.org/entity/Q17114254|http:...,"Aarvik, L.",Leif Aarvik|Lars Aarvik


In [38]:
this_output_file='data/plazi_collectors_matches_wikidata_items_group_concat_%s.csv' % (
    # "20230531_1156"
    datetime.today().strftime('%Y%m%d') # '%Y%m%d_%H%M'
)

# collectors_matches_group.to_csv(this_output_file_name)
collectors_matches_group.to_csv(this_output_file)

print("Wrote groups of collectors matches into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote groups of collectors matches into data/plazi_collectors_matches_wikidata_items_group_concat_20230719.csv (85272 kB)


Get individual WikiData items (TODO review code): 
- associate collector name match + individual WikiData items (remember: we matched the `canonical_string`)

In [39]:
# TODO get  list of atomized collectors matches down to single wikidata items
collectors_matches_t1 = pd.merge(
    collectors_matches, wikidata, 
    left_on='namematch_wikidata', right_on='canonical_string'
)
# collectors_matches_t1.drop(columns=['canonical_string'])

# link counts of wikidata items with same canonical name string
collectors_matches_t2 = pd.merge(
    collectors_matches_t1, wikidata, 
    left_on="namematch_wikidata", right_on="canonical_string"
    , suffixes=('__collmatches', '__wdata-isolated') # append to left-data, right-data only when identical column names occur
)

# TODO AP: add count of duplicates?

print(list(collectors_matches_t2.columns))
# ['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick', 
# 'appellation', 'title', 'DocCount', 'MatCitId', 'MatCitGbifOccurrenceId', 
# 'MatCitDate', 'MatCitDecade', 'MatCitYear', 'MatCitMonth', 
# 'canonical_string_collector_parsed', 'index', 'namematch_collector', 
# 'namematch_wikidata', 'namematch_distance', 'item__collmatches', 
# 'itemLabel__collmatches', 'surname__collmatches', 'initials__collmatches', 
# 'canonical_string__collmatches', 'orcid__collmatches', 'viaf__collmatches', 
# 'isni__collmatches', 'harv__collmatches', 'ipni__collmatches', 
# 'abbr__collmatches', 'bionomia_id__collmatches', 'yob__collmatches', 
# 'yod__collmatches', 'wyb__collmatches', 'wye__collmatches', 
# 'item__wdata-isolated', 'itemLabel__wdata-isolated', 'surname__wdata-isolated', 
# 'initials__wdata-isolated', 'canonical_string__wdata-isolated', 
# 'orcid__wdata-isolated', 'viaf__wdata-isolated', 'isni__wdata-isolated', 
# 'harv__wdata-isolated', 'ipni__wdata-isolated', 'abbr__wdata-isolated', 
# 'bionomia_id__wdata-isolated', 'yob__wdata-isolated', 'yod__wdata-isolated', 
# 'wyb__wdata-isolated', 'wye__wdata-isolated']

['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick', 'appellation', 'title', 'canonical_string_collector_parsed', 'DocCount', 'MatCitId', 'MatCitGbifOccurrenceId', 'MatCitDate', 'MatCitDecade', 'MatCitYear', 'MatCitMonth', 'index', 'namematch_collector', 'namematch_wikidata', 'namematch_distance', 'item__collmatches', 'itemLabel__collmatches', 'surname__collmatches', 'initials__collmatches', 'canonical_string__collmatches', 'orcid__collmatches', 'viaf__collmatches', 'isni__collmatches', 'harv__collmatches', 'ipni__collmatches', 'abbr__collmatches', 'bionomia_id__collmatches', 'yob__collmatches', 'yod__collmatches', 'wyb__collmatches', 'wye__collmatches', 'item__wdata-isolated', 'itemLabel__wdata-isolated', 'surname__wdata-isolated', 'initials__wdata-isolated', 'canonical_string__wdata-isolated', 'orcid__wdata-isolated', 'viaf__wdata-isolated', 'isni__wdata-isolated', 'harv__wdata-isolated', 'ipni__wdata-isolated', 'abbr__wdata-isolated', 'bionomia_id__wdata-isolated',

In [40]:
collectors_matches_t2.sort_values(by=['namematch_distance', 'canonical_string_collector_parsed','MatCitDate'], inplace=True)
collectors_matches_t2.drop_duplicates(inplace=True)
collectors_matches_t2.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,DocCount,...,viaf__wdata-isolated,isni__wdata-isolated,harv__wdata-isolated,ipni__wdata-isolated,abbr__wdata-isolated,bionomia_id__wdata-isolated,yob__wdata-isolated,yod__wdata-isolated,wyb__wdata-isolated,wye__wdata-isolated
843717,Aagaard,K.,NaN,NaN,NaN,NaN,NaN,NaN,"Aagaard, K.",1,...,273390791,NaN,NaN,NaN,NaN,Q55216516,1947.0,NaN,NaN,NaN
843718,Aagaard,K.,NaN,NaN,NaN,NaN,NaN,NaN,"Aagaard, K.",1,...,273390791,NaN,NaN,NaN,NaN,Q55216516,1947.0,NaN,NaN,NaN
24113,Aarvik',L.,NaN,NaN,NaN,NaN,NaN,NaN,"Aarvik', L.",1,...,14511016,0000 0000 4817 3374,NaN,NaN,NaN,0000-0002-0112-8837,1954.0,NaN,NaN,NaN
24114,Aarvik',L.,NaN,NaN,NaN,NaN,NaN,NaN,"Aarvik', L.",1,...,NaN,NaN,NaN,NaN,NaN,Q106823278,1892.0,1981.0,NaN,NaN
24115,Aarvik',L.,NaN,NaN,NaN,NaN,NaN,NaN,"Aarvik', L.",1,...,14511016,0000 0000 4817 3374,NaN,NaN,NaN,0000-0002-0112-8837,1954.0,NaN,NaN,NaN


In [41]:
this_output_file='data/plazi_collectors_matches_wikidata-botanists_all-columns_%s.csv' % (
    # "20230531"
    datetime.today().strftime('%Y%m%d') # '%Y%m%d_%H%M'
)

collectors_matches_t2.to_csv(this_output_file)

print("Wrote isolated WikiData items of collector matches into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote isolated WikiData items of collector matches into data/plazi_collectors_matches_wikidata-botanists_all-columns_20230719.csv (326244 kB)


In [ ]:
# TODO further evaluation or filtering, counting, clean up aso.

TODO document columns

Explanation of columns:

Column | Description
-|-
**Botanical collectors** |
family | parsed family name
given | parsed given name
TODO … | Year of first collection
TODO end_date | Year of last collection
TODO activity_span | Number of years between first and last collection
**Name matching** |
nammatch_collector | matched name of the data set
nammatch_wikidata | matched name; = Wikidata item label name is matched to
name_match_distance | Nearest Neighbour distance between the name and matched name; the lower the value, the better the match
**Wikidata** |
item | Wikidata Item ID (URL)
itemLabel | Wikidata Item label
surname	| Surname; derived from item label
initials | Initials; derived from item label
canonical_string | Canonical name string; derived from item label, used for matching
orcid | ORCID ([P496](https://www.wikidata.org/wiki/Property:P496))
viaf | VIAF ID ([P214](https://www.wikidata.org/wiki/Property:P214))
isni | ISNI ID ([P213](https://www.wikidata.org/wiki/Property:P496))	
harv | Harvard Index of Botanists ID ([P6264](https://www.wikidata.org/wiki/Property:P6264))
ipni | IPNI author ID ([P586](https://www.wikidata.org/wiki/Property:P586))
abbr | botanist author abbreviation (standard form) ([P428](https://www.wikidata.org/wiki/Property:P428))
yob	| Year of birth (derived from [P569](https://www.wikidata.org/wiki/Property:P569))
yod	| Year of death (derived from [P496](https://www.wikidata.org/wiki/Property:P570))
wyb	| Start year of work period ([P2031](https://www.wikidata.org/wiki/Property:P2031))
wye | End year of work period ([P2032](https://www.wikidata.org/wiki/Property:P2032))